In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline 

pd.set_option('display.max_columns', None)

In [2]:
train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")
sub = pd.read_csv("sub.csv")

In [3]:
train['train_or_test']='train'
test['train_or_test']='test'
df=pd.concat([train,test])

C:\Users\ABetta\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
train.head()

enrollee_id      city  city_development_index gender  \
0        23798  city_149                   0.689   Male   
1        29166   city_83                   0.923   Male   
2           46   city_16                   0.910    NaN   
3        18527   city_64                   0.666   Male   
4        21751  city_100                   0.887    NaN   

       relevent_experience enrolled_university education_level  \
0  Has relevent experience       no_enrollment        Graduate   
1  Has relevent experience       no_enrollment        Graduate   
2  Has relevent experience       no_enrollment        Graduate   
3  Has relevent experience       no_enrollment        Graduate   
4   No relevent experience       no_enrollment         Masters   

  major_discipline experience company_size    company_type last_new_job  \
0             STEM          3      100-500         Pvt Ltd            1   
1             STEM         14          <10  Funded Startup            1   
2             STEM          6        50-99   Public Sector            2   
3             STEM         14        50-99         Pvt Ltd            1   
4             STEM          8          NaN             NaN            2   

   training_hours  target train_or_test  
0             106       0         train  
1              69       0         train  
2               4       0         train  
3              26       0         train  
4              88       1         train

In [5]:
df.isnull().sum()

city                          0
city_development_index        0
company_size               8830
company_type               9369
education_level             852
enrolled_university         621
enrollee_id                   0
experience                  103
gender                     7486
last_new_job                671
major_discipline           5231
relevent_experience           0
target                    15021
train_or_test                 0
training_hours                0
dtype: int64

In [6]:
cols_with_missing = [col for col in df.columns
                     if df[col].isnull().any()]
for col in cols_with_missing:
    df[col + '_was_missing'] = df[col].isnull()

In [7]:
df.reset_index(inplace=True)

In [8]:
def fast_mode(df, key_cols, value_col):
    """ 
    Calculate a column mode, by group, ignoring null values. 

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame over which to calcualate the mode. 
    key_cols : list of str
        Columns to groupby for calculation of mode.
    value_col : str
        Column for which to calculate the mode. 

    Return
    ------ 
    pandas.DataFrame
        One row for the mode of value_col per key_cols group. If ties, 
        returns the one which is sorted first. 
    """
    return (df.groupby(key_cols + [value_col]).size() 
              .to_frame('counts').reset_index() 
              .sort_values('counts', ascending=False) 
              .drop_duplicates(subset=key_cols)).drop(columns='counts')

In [9]:
df.loc[df.company_size.isnull(), 'company_size'] = df.company_type.map(fast_mode(df, ['company_type'], 'company_size').set_index('company_type').company_size)

#df.loc[df.company_type.isnull(), 'company_type'] = df.company_size.map(fast_mode(df, ['company_size'], 'company_type').set_index('company_size').company_type)

#df.loc[df.education_level.isnull(), 'education_level'] = df.experience.map(fast_mode(df, ['experience'], 'education_level').set_index('experience').education_level)

#df.loc[df.enrolled_university.isnull(), 'enrolled_university'] = df.experience.map(fast_mode(df, ['experience'], 'enrolled_university').set_index('experience').enrolled_university)

#df.loc[df.experience.isnull(), 'experience'] = df.education_level.map(fast_mode(df, ['education_level'], 'experience').set_index('education_level').experience)

#df.loc[df.last_new_job.isnull(), 'last_new_job'] = df.experience.map(fast_mode(df, ['experience'], 'last_new_job').set_index('experience').last_new_job)





In [10]:
df['gender'].fillna(df['gender'].mode()[0], inplace=True)
df['enrolled_university'].fillna(df['enrolled_university'].mode()[0], inplace=True)
df['education_level'].fillna(df['education_level'].mode()[0], inplace=True)
df['major_discipline'].fillna(df['major_discipline'].mode()[0], inplace=True)
df['experience'].fillna(df['company_type'].mode()[0], inplace=True)
df['company_type'].fillna(df['company_type'].mode()[0], inplace=True)
df['company_size'].fillna(df['company_size'].mode()[0], inplace=True)
df['last_new_job'].fillna(df['last_new_job'].mode()[0], inplace=True)

In [11]:
df['company_size'] = df['company_size'].map({'<10': 0,
                                                      '10/49': 1,
                                                      '50-99': 2,
                                                      '100-500': 3,
                                                      '500-999': 4,
                                                      '1000-4999': 5,
                                                      '5000-9999': 6,
                                                      '10000+': 7})

df['experience'] = df['experience'].map({'<1': 0,
                                                  '1': 1,
                                                  '2': 2,
                                                  '3': 3,
                                                  '4': 4,
                                                  '5': 5,
                                                  '6': 6,
                                                  '7': 7,
                                                  '8': 8,
                                                  '9': 9,
                                                  '10': 10,
                                                  '11': 11,
                                                  '12': 12,
                                                  '13': 13,
                                                  '14': 14,
                                                  '15': 15,
                                                  '16': 16,
                                                  '17': 17,
                                                  '18': 18,
                                                  '19': 19,
                                                  '20': 20,
                                                  '>20': 21,
                                                  'Pvt Ltd' :22
                                                 })


df['last_new_job'] = df['last_new_job'].map({'never': 0,
                                                      '1': 1,
                                                      '2': 2,
                                                      '3': 3,
                                                      '4': 4,
                                                      '>4': 5})

In [12]:
train=df.loc[df.train_or_test.isin(['train'])]
test=df.loc[df.train_or_test.isin(['test'])]
train.drop(columns={'train_or_test'},axis=1,inplace=True)
test.drop(columns={'train_or_test'},axis=1,inplace=True)

C:\Users\ABetta\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [13]:
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None, 
                  tst_series=None, 
                  target=None, 
                  min_samples_leaf=1, 
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior  
    """ 
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

In [14]:
train['education_level'].value_counts()

Graduate          11226
Masters            4319
High School        2032
Phd                 459
Primary School      323
Name: education_level, dtype: int64

In [15]:
map={'Other':0,'Feamle':1,'Male':2}
train['gender_me']=train['gender'].map(map)
test['gender_me']=test['gender'].map(map)

C:\Users\ABetta\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\ABetta\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
tr_g, te_g = target_encode(train["city"], 
                         test["city"], 
                         target=train.target, 
                         min_samples_leaf=100,
                         smoothing=10,
                         noise_level=0.01)
train['city_me']=tr_g
test['city_me']=te_g

C:\Users\ABetta\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\ABetta\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [17]:
train.columns

Index(['index', 'city', 'city_development_index', 'company_size',
       'company_type', 'education_level', 'enrolled_university', 'enrollee_id',
       'experience', 'gender', 'last_new_job', 'major_discipline',
       'relevent_experience', 'target', 'training_hours',
       'company_size_was_missing', 'company_type_was_missing',
       'education_level_was_missing', 'enrolled_university_was_missing',
       'experience_was_missing', 'gender_was_missing',
       'last_new_job_was_missing', 'major_discipline_was_missing',
       'target_was_missing', 'gender_me', 'city_me'],
      dtype='object')

In [18]:
x=train[[#'city', 
    'city_development_index', 'company_size',
       'company_type', 'education_level', 
         'enrolled_university',
       'experience', 'gender',
         'last_new_job', 'major_discipline',
       'relevent_experience',
         'training_hours',
       'company_size_was_missing', 'company_type_was_missing',
       'education_level_was_missing', 'enrolled_university_was_missing',
       'experience_was_missing', 'gender_was_missing',
       'last_new_job_was_missing', 'major_discipline_was_missing','city_me'
        ]]

y=train.loc[:,['target']]

test=test[[#'city', 
           'city_development_index', 'company_size',
       'company_type', 'education_level', 
           'enrolled_university',
       'experience', 'gender',
           'last_new_job', 'major_discipline',
       'relevent_experience',
           'training_hours',
       'company_size_was_missing', 'company_type_was_missing',
       'education_level_was_missing', 'enrolled_university_was_missing',
       'experience_was_missing', 'gender_was_missing',
       'last_new_job_was_missing', 'major_discipline_was_missing','city_me'
          ]]


In [19]:
from catboost import CatBoostClassifier,Pool
from sklearn.model_selection import train_test_split

In [20]:
x_train,x_valid,y_train,y_valid=train_test_split(x,y,train_size =0.8)

In [21]:
x_train.dtypes

city_development_index             float64
company_size                         int64
company_type                        object
education_level                     object
enrolled_university                 object
experience                           int64
gender                              object
last_new_job                         int64
major_discipline                    object
relevent_experience                 object
training_hours                       int64
company_size_was_missing              bool
company_type_was_missing              bool
education_level_was_missing           bool
enrolled_university_was_missing       bool
experience_was_missing                bool
gender_was_missing                    bool
last_new_job_was_missing              bool
major_discipline_was_missing          bool
city_me                            float64
dtype: object

In [58]:
cat_features=[2,3,4,6,8,9]#,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41] 

In [23]:
train_pool=Pool(x, y, cat_features=cat_features)
test_pool=Pool(test, cat_features =cat_features) 

In [24]:
weight=[1, 0.43]

In [25]:
model = CatBoostClassifier(
        iterations=5000,logging_level="Silent",early_stopping_rounds=500,
    use_best_model=True,custom_loss=["AUC"], class_weights=weight, 
        eval_metric="AUC")

In [26]:
model.fit(  x_train,y_train, 
        cat_features=cat_features,
        eval_set=(x_valid,y_valid),
        plot=True 
        )

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [27]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_valid, model.predict_proba(x_valid) [:,1]) 

0.659659030720339

In [28]:
feature_importances=model.get_feature_importance(train_pool) 
feature_names=x.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

city_me: 18.896401907224
city_development_index: 11.437743387844868
experience: 8.943939202760847
company_type_was_missing: 8.701357652345562
company_size_was_missing: 8.495776306019398
last_new_job: 7.556085763375091
major_discipline_was_missing: 6.613533020638975
relevent_experience: 4.828592320149441
company_size: 4.107019819171324
enrolled_university: 4.044402783688551
training_hours: 3.90693795132463
education_level: 2.389665798973756
education_level_was_missing: 2.2441697246667593
company_type: 1.965409612454023
gender_was_missing: 1.9211872432888124
gender: 1.6155630242070975
enrolled_university_was_missing: 0.8285217665032267
major_discipline: 0.6839201570218201
experience_was_missing: 0.46752206536252466
last_new_job_was_missing: 0.35225049297928734


In [33]:
final_model = CatBoostClassifier(
        iterations=1500,logging_level="Silent",class_weights=weight, random_seed=0,
        eval_metric="AUC")
final_model.fit(x,y, cat_features=cat_features )

In [34]:
pred=final_model.predict_proba(test)[:,1]

In [35]:
sub['target']=pred

In [36]:
sub.to_csv('catboostbaseline2.csv',index=False)

In [49]:
cat=pd.read_csv('catboostbaseline2.csv')
#cat1=pd.read_csv('catboost1.csv')

In [50]:
xgb=pd.read_csv('xgb.csv')

In [51]:
sub['target']=0.8*cat.target+0.2*xgb.target

In [52]:
sub.to_csv('ensemble.csv',index=False)